# Mini-analisi di DMR (Differentially Methylated Regions)

Obiettivi del notebook:
- Simulare CpG lungo un tratto genomico (es. un promotore).
- Calcolare la differenza di metilazione per CpG tra controlli ed esposti.
- Identificare in modo semplice regioni con differenza coerente (DMR toy).

_Esempio didattico, non un pipeline reale._

In [ ]:
from __future__ import annotations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
np.random.seed(123)

## Simulazione di CpG lungo un segmento genomico

Immaginiamo un tratto di 10 kb con 50 CpG.
- Nei controlli la metilazione è ~0.4.
- Nei campioni esposti, in una regione centrale (es. 4 kb–6 kb) la metilazione è ~0.7.
Questo creerà una **DMR ipermetilata** nel gruppo esposto.

In [ ]:
n_cpg = 50
n_ctrl = 15
n_exp = 15

positions = np.linspace(0, 10_000, n_cpg, dtype=int)  # 0-10kb
cpg_ids = [f"cgDMR_{i+1}" for i in range(n_cpg)]
sample_ids = [f"CTRL_{i+1}" for i in range(n_ctrl)] + [f"EXP_{i+1}" for i in range(n_exp)]
group = np.array([0]*n_ctrl + [1]*n_exp)
group_series = pd.Series(group, index=sample_ids, name="group")

# Metilazione di base ~0.4 per tutti
beta_matrix = np.random.normal(loc=0.4, scale=0.05, size=(n_cpg, n_ctrl + n_exp))

# Regione DMR ipermetilata tra 4000 e 6000 bp nel gruppo esposto
dmr_mask = (positions >= 4000) & (positions <= 6000)
for i, is_dmr in enumerate(dmr_mask):
    if is_dmr:
        beta_matrix[i, group == 1] = np.random.normal(loc=0.7, scale=0.05, size=(group == 1).sum())

beta_matrix = np.clip(beta_matrix, 0, 1)
beta_df = pd.DataFrame(beta_matrix, index=cpg_ids, columns=sample_ids)

region_df = pd.DataFrame({
    "CpG": cpg_ids,
    "position": positions,
}).set_index("CpG")
region_df.head()

## Differenza di metilazione per CpG

Calcoliamo, per ogni CpG in questo tratto:
- media nei controlli
- media negli esposti
- differenza (EXP - CTRL)
- p-value da t-test

In [ ]:
results = []
for cpg in cpg_ids:
    vals = beta_df.loc[cpg, :]
    vals_ctrl = vals[group_series == 0]
    vals_exp = vals[group_series == 1]
    diff = vals_exp.mean() - vals_ctrl.mean()
    tstat, pval = stats.ttest_ind(vals_exp, vals_ctrl, equal_var=False)
    results.append({
        "CpG": cpg,
        "position": int(region_df.loc[cpg, "position"]),
        "mean_ctrl": vals_ctrl.mean(),
        "mean_exp": vals_exp.mean(),
        "diff_exp_minus_ctrl": diff,
        "pval": pval,
    })

res_df = pd.DataFrame(results).set_index("CpG")
res_df.head()

## Visualizzare la differenza lungo il segmento genomico

Mettiamo la differenza di metilazione (EXP - CTRL) in funzione della posizione.

In [ ]:
plt.plot(res_df["position"], res_df["diff_exp_minus_ctrl"], marker="o")
plt.axhline(0, linestyle="--")
plt.xlabel("Posizione (bp)")
plt.ylabel("Diff metilazione (EXP - CTRL)")
plt.title("Profilo di differenza di metilazione lungo il tratto")
plt.show()

Si vede chiaramente una regione centrale con differenza positiva (~ipermetilata negli esposti).

## Identificare una DMR in modo semplice

Definiamo una "DMR" toy come una sequenza di CpG adiacenti in cui:
- la differenza media è ≥ 0.2
- i p-value sono ≤ 0.01
- le CpG sono contigue in posizione.

In [ ]:
threshold_diff = 0.2
threshold_p = 0.01

candidate = res_df[(res_df["diff_exp_minus_ctrl"].abs() >= threshold_diff) & (res_df["pval"] <= threshold_p)].copy()
candidate = candidate.sort_values("position")
candidate

Cerchiamo ora blocchi contigui di CpG candidate (in base alla posizione).

In [ ]:
regions = []
if not candidate.empty:
    current_start = candidate["position"].iloc[0]
    current_end = candidate["position"].iloc[0]
    current_cpgs = [candidate.index[0]]
    current_diffs = [candidate["diff_exp_minus_ctrl"].iloc[0]]

    for cpg, row in candidate.iloc[1:].iterrows():
        pos = row["position"]
        diff = row["diff_exp_minus_ctrl"]
        # se la distanza è <= 300 bp, consideriamola contigua (soglia arbitraria)
        if pos - current_end <= 300:
            current_end = pos
            current_cpgs.append(cpg)
            current_diffs.append(diff)
        else:
            regions.append({
                "start": current_start,
                "end": current_end,
                "n_cpg": len(current_cpgs),
                "mean_diff": np.mean(current_diffs),
                "CpGs": current_cpgs,
            })
            current_start = pos
            current_end = pos
            current_cpgs = [cpg]
            current_diffs = [diff]

    # aggiungi l'ultima regione
    regions.append({
        "start": current_start,
        "end": current_end,
        "n_cpg": len(current_cpgs),
        "mean_diff": np.mean(current_diffs),
        "CpGs": current_cpgs,
    })

regions_df = pd.DataFrame(regions)
regions_df

Queste sono le nostre **DMR toy**.

## Visualizzare la regione DMR sul profilo di differenza

Evidenziamo la prima regione (se esiste) sopra il profilo di differenza.

In [ ]:
plt.plot(res_df["position"], res_df["diff_exp_minus_ctrl"], marker="o")
plt.axhline(0, linestyle="--")
plt.xlabel("Posizione (bp)")
plt.ylabel("Diff metilazione (EXP - CTRL)")
plt.title("Profilo di differenza + regione DMR toy")

if not regions_df.empty:
    r0 = regions_df.iloc[0]
    plt.axvspan(r0["start"], r0["end"], alpha=0.2)

plt.show()

In analisi reali, i metodi per la ricerca di **DMR** (es. `methylKit`, `DSS`, `DMRcaller`) usano modelli statistici avanzati e considerano la correlazione tra CpG vicine.

Qui abbiamo costruito un semplice esempio per collegare:
- CpG singole (DMC)
- regioni di CpG con differenza coerente (DMR)

adatto per discussioni in aula su come l'ambiente possa rimodellare intere regioni regolatorie.